<a href="https://colab.research.google.com/github/yeyekang/Comparison-of-Multi-task-Models/blob/main/shared_bottom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_openml
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

SEED = 1
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


# ==== 回调函数：输出ROC-AUC ====
class ROCCallback(Callback):
    def __init__(self, train, val, test):
        self.train_X, self.train_Y = train
        self.val_X, self.val_Y = val
        self.test_X, self.test_Y = test

    def on_epoch_end(self, epoch, logs=None):
        train_pred = self.model.predict(self.train_X, verbose=0)
        val_pred = self.model.predict(self.val_X, verbose=0)
        test_pred = self.model.predict(self.test_X, verbose=0)

        for i, name in enumerate(self.model.output_names):
            tr_auc = roc_auc_score(self.train_Y[i], train_pred[i])
            va_auc = roc_auc_score(self.val_Y[i], val_pred[i])
            te_auc = roc_auc_score(self.test_Y[i], test_pred[i])
            print(f"[Epoch {epoch+1}] {name} - AUC: Train={tr_auc:.4f}, Val={va_auc:.4f}, Test={te_auc:.4f}")


# ==== 数据准备 ====
def data_preparation():
    # 加载 Adult 数据集
    data = fetch_openml("adult", version=2, as_frame=True)
    df = data.frame

    # 特征 & 标签
    X = pd.get_dummies(df.drop(columns=["class", "marital-status"]))
    y_income = (df["class"] == ">50K").astype(int).values
    y_marital = (df["marital-status"] == "Never-married").astype(int).values

    # 划分 train/val/test (70/15/15)
    n = len(X)
    idx = np.arange(n)
    np.random.shuffle(idx)
    train_end, val_end = int(0.7 * n), int(0.85 * n)

    train_idx = idx[:train_end]
    val_idx = idx[train_end:val_end]
    test_idx = idx[val_end:]

    X_train = X.iloc[train_idx].values.astype(np.float32)
    X_val = X.iloc[val_idx].values.astype(np.float32)
    X_test = X.iloc[test_idx].values.astype(np.float32)

    y_train = [to_categorical(y_income[train_idx]), to_categorical(y_marital[train_idx])]
    y_val = [to_categorical(y_income[val_idx]), to_categorical(y_marital[val_idx])]
    y_test = [to_categorical(y_income[test_idx]), to_categorical(y_marital[test_idx])]

    output_info = [(2, "income"), (2, "marital")]

    return X_train, y_train, X_val, y_val, X_test, y_test, output_info


# ==== Shared-Bottom 模型 ====
def build_shared_bottom(num_features, output_info):
    inputs = Input(shape=(num_features,), name="input_layer")

    # shared-bottom 层
    shared = Dense(64, activation="relu", kernel_initializer=VarianceScaling())(inputs)
    shared = Dense(32, activation="relu", kernel_initializer=VarianceScaling())(shared)

    outputs = []
    for units, name in output_info:
        tower = Dense(16, activation="relu", kernel_initializer=VarianceScaling())(shared)
        out = Dense(units, activation="softmax", name=name, kernel_initializer=VarianceScaling())(tower)
        outputs.append(out)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
    optimizer=Adam(),
    loss={name: "binary_crossentropy" for _, name in output_info},
    metrics={name: ["accuracy"] for _, name in output_info}
)

    return model


# ==== 主函数 ====
def main():
    X_train, y_train, X_val, y_val, X_test, y_test, output_info = data_preparation()

    print("Training data shape:", X_train.shape)
    print("Validation data shape:", X_val.shape)
    print("Test data shape:", X_test.shape)

    model = build_shared_bottom(X_train.shape[1], output_info)
    model.summary()

    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        callbacks=[ROCCallback((X_train, y_train), (X_val, y_val), (X_test, y_test))],
        batch_size=128
    )


if __name__ == "__main__":
    main()


Training data shape: (34189, 98)
Validation data shape: (7326, 98)
Test data shape: (7327, 98)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 98)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      6,336 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 16)        │        528 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 16)        │        528 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ income (Dense)      │ (None, 2)         │         34 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ marital (Dense)     │ (None, 2)         │         34 │ dense_11[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,540 (37.27 KB)

 Trainable params: 9,540 (37.27 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - income_accuracy: 0.6813 - income_loss: 332.8875 - loss: 461.4797 - marital_accuracy: 0.6256 - marital_loss: 128.5909[Epoch 1] income - AUC: Train=0.6081, Val=0.6009, Test=0.6106
[Epoch 1] marital - AUC: Train=0.5118, Val=0.5109, Test=0.5136
268/268 ━━━━━━━━━━━━━━━━━━━━ 14s 40ms/step - income_accuracy: 0.6814 - income_loss: 332.0294 - loss: 460.3374 - marital_accuracy: 0.6255 - marital_loss: 128.3053 - val_income_accuracy: 0.7882 - val_income_loss: 19.0310 - val_loss: 31.2372 - val_marital_accuracy: 0.6706 - val_marital_loss: 12.3391
Epoch 2/10
264/268 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - income_accuracy: 0.7315 - income_loss: 26.7723 - loss: 43.3204 - marital_accuracy: 0.5860 - marital_loss: 16.5481[Epoch 2] income - AUC: Train=0.5832, Val=0.5841, Test=0.5888
[Epoch 2] marital - AUC: Train=0.5461, Val=0.5394, Test=0.5421
268/268 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - income_accuracy: 0.7313 - income_loss: 26.7924 - loss: 43.3431 - marita